# Train a Simple TensorFlow Lite for Microcontrollers model

This notebook demonstrates the process of training model using TensorFlow and converting it for use with TensorFlow Lite for Microcontrollers. 



## Create Model


Import Dependencies

In [ ]:
# TensorFlow is an open source machine learning library
import tensorflow as tf
# Keras is TensorFlow's high-level API for deep learning
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input 

# Numpy is a math library
import numpy as np

# Pandas is a data manipulation library 
import pandas as pd



### Create a dataset to train our model
We'll create a python generator and feed that through a tensorflow Dataset to train our model

In [ ]:
def data_generator():
    while(True):
        number1 = np.random.uniform();
        number2 = np.random.uniform();
        # our input data is an array containing 2 numbers
        X = [number1, number2]
        # our label is 1 or 0
        Y = 1 if number2 > number1 else 0
        # our generator should return the input data and the label
        yield X, [Y]
        
# create a dataset from our generator
train_dataset = tf.data.Dataset.from_generator(
    data_generator, 
    output_types = (tf.float32, tf.int32),
    output_shapes=((2), (1))
)
train_dataset = train_dataset.batch(batch_size=30)

### Our very simple mode

We don't need a very complicated model for our problem, so we'll just define a small neural network with an input layer and an output layer.

It's important that the activation function for the output should be sigmoid. This activation function will output a value between 0 and 1.

In [ ]:
model = Sequential([
    Input(shape=(2)),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 15        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________


### Compile our model
For our loss function we need to use BinaryCrossentropy.

Crossentropy quantifies the difference between two probability distribution.

We have a binary distribution (True or False) so we use binary crossentropy to compare the output from our model with the true distribution.

In [ ]:
# Compile the model using the standard 'adam' optimizer
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

### Train the model

In [ ]:
# Train the model

model.fit(
    train_dataset,
    steps_per_epoch=1000,
    epochs=4
)

Epoch 1/4
1000/1000 [==============================] - 7s 6ms/step - loss: 0.5482 - accuracy: 0.7717
Epoch 2/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.2223 - accuracy: 0.9663
Epoch 3/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1294 - accuracy: 0.9936
Epoch 4/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0990 - accuracy: 0.9951


### Testing our model
We can feed in some values and see what our model predicts

In [ ]:
test_X = np.array([
    [100.0, 0.2],
    [-0.3, 0.4],
    [0.2, 0.1],
    [0.1, 0.004]
])
Y = model.predict_on_batch(test_X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(Y)


[[0.00]
 [1.00]
 [0.12]
 [0.13]]


## Generate a TensorFlow Model

### Configure Defaults



In [ ]:
# Define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

In [ ]:
# Save the model to disk
model.save(MODEL_TF)

INFO:tensorflow:Assets written to: models/model/assets



## Generate a TensorFlow Lite Model

We now have an acceptably accurate model. We'll use the TensorFlow Lite Converter to convert the model into a special, space-efficient format for use on memory-constrained devices.

Since this model is going to be deployed on a microcontroller, we want it to be as tiny as possible! 

In the following cell, we'll convert the model twice: once with quantization, once without.

In [ ]:
# Convert the model to the TensorFlow Lite format 
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_no_quant_tflite)



1432

In [ ]:
# Calculate size
size_tf = os.path.getsize(MODEL_TF)
size_tflite = os.path.getsize(MODEL_TFLITE)

In [ ]:
# Compare size
pd.DataFrame.from_records(
    [["TensorFlow", f"{size_tf} bytes", ""],
     ["TensorFlow Lite", f"{size_tflite} bytes ", f"(reduced by {size_tf - size_tflite} bytes)"]],
     columns = ["Model", "Size", ""], index="Model")

,Size,
Model,,
TensorFlow,4096 bytes,
TensorFlow Lite,1432 bytes,(reduced by 2664 bytes)


### Generate a TensorFlow Lite for Microcontrollers Model
Convert the TensorFlow Lite  model into a C source file that can be loaded by TensorFlow Lite for Microcontrollers.


In [ ]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
H

## Deploy to a Microcontroller

f you have generated a new model, then update the values assigned to the variables defined in [`hello_world/model.cc`](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/hello_world/model.cc) with values displayed after running the following cell.

In [ ]:
# Print the C source file
!cat {MODEL_TFLITE_MICRO}


unsigned char models_model_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00,
  0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x18, 0x00, 0x1c, 0x00, 0x14, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x18, 0x00, 0x00, 0x00, 0x20, 0x00, 0x00, 0x00, 0x2c, 0x01, 0x00, 0x00,
  0x20, 0x00, 0x00, 0x00, 0xd4, 0x00, 0x00, 0x00, 0x44, 0x00, 0x00, 0x00,
  0x02, 0x00, 0x00, 0x00, 0x18, 0x03, 0x00, 0x00, 0xc8, 0x01, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x30, 0x01, 0x00, 0x00, 0x0a, 0x00, 0x00, 0x00,
  0x40, 0x05, 0x00, 0x00, 0x3c, 0x05, 0x00, 0x00, 0x78, 0x04, 0x00, 0x00,
  0x18, 0x04, 0x00, 0x00, 0xa8, 0x03, 0x00, 0x00, 0x4c, 0x03, 0x00, 0x00,
  0x28, 0x05, 0x00, 0x00, 0x24, 0x05, 0x00, 0x00, 0x20, 0x05, 0x00, 0x00,
  0xc4, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00,
  0x0c, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x08, 0x00, 0x04, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x10, 0x00, 0x00, 0x00, 0x18, 0x00, 0x00, 0x00